## Directional DIC showcase
More information on the method can be found in the corresponding paper:
Masmeijer, Thijs and Zaletelj, Klemen and Slavič, Janko and Habtour, Ed, Directional DIC Method with Automatic Feature Selection. Available at SSRN: https://ssrn.com/abstract=4907539 or http://dx.doi.org/10.2139/ssrn.4907539

In [21]:
import os
import sys
sys.path.insert(0, os.path.realpath('__file__'))

import numpy as np
import matplotlib.pyplot as plt
import pyidi
from scipy.ndimage import uniform_filter

The Directional DIC method is demonstrated on some synthetic data. Directional DIC is recommended for use on cases where vibration is predomentely uni-directional, but still works on 2D cases.

In [22]:
filename = 'data/data_synthetic.cih'
video = pyidi.pyIDI(filename)

Watch the video:

In [ ]:
%matplotlib qt
import matplotlib.animation as animation

def play_video(frame_range = None, interval=30, points = None, ij_counter = (0.65, 0.05)):
    
    """
    Plays the video from the given video object.
    Args:
        frame_range (range object): The range of frames to play.
        interval (int): The interval between frames in milliseconds.
        points (ndarray): Optional tracked points to plot on the video.
        ij_counter (tuple): The position of the frame counter.
    """
    if frame_range is None:
        frame_range = range(0, video.info['Total Frame'])
    fig, ax = plt.subplots()
    im = ax.imshow(video.mraw[frame_range[0]], cmap='gray')
    text = ax.text(ij_counter[0], ij_counter[1], '', transform=ax.transAxes, color='black', ha='right', va='bottom')

    if points is not None:
        pts = ax.plot(points[:,0,1], points[:,0,0], 'r.')

    def update(i):
        im.set_data(video.mraw[i])
        text.set_text(f'Frame {i}')
        if points is not None:
            pts[0].set_data(points[:,i,1], points[:,i,0])
        return im, text

    ani = animation.FuncAnimation(fig, update, frames=frame_range, interval=interval)
    plt.show()
    return ani

ani = play_video(range(1, video.info['Total Frame']))

We will first use the conventional `DIC` method as a reference

In [ ]:
N_tracking_points = 1
tol = 1e-8
roi_size = (11,11)
video.set_method('lk') # Set the method to Lucas-Kanade (or DIC)
video.method.configure(reference_image = (0,100), resume_analysis = False, tol=tol, roi_size=roi_size)


# DIC is expected to work will on image subsets with high contrast in two orthogonal directions. We can find the best subsets according to:
reference_image = video.method._set_reference_image(video, video.method.reference_image)
Gi, Gj = np.gradient(reference_image)
G = np.sqrt(Gi**2 + Gj**2)
G_filtered = uniform_filter(G, size=roi_size)
points2d = np.array(np.unravel_index(np.argsort(G_filtered.flatten())[-N_tracking_points:], G_filtered.shape)).T  # Find the N_tracking_points highest gradient points

fig, ax = plt.subplots()
ax.imshow(G_filtered, cmap='gray')
plt.show()

video.set_points(points2d)
video.show_points()
displacements_2d = video.get_displacements(processes=1, autosave=False)

Next, we will use the `Directional DIC` method. Directional DIC uses a predetermined motion direction. Consequently, Directional DIC only requires a high gradient in the direction of motion to track well.

In [ ]:
video = pyidi.pyIDI(filename)
video.set_method('lk_1D')
'''
 Here we define the direction of the motion.
'''
dij = (1, 0) # vertical direction
roi_size = (3, 7) 
video.method.configure(reference_image = (0, 100), dij=dij, resume_analysis = False, tol=tol,roi_size=roi_size)
reference_image = video.method._set_reference_image(video, video.method.reference_image)
Gi, Gj = np.gradient(reference_image)
G = np.abs(Gi*dij[0] + Gj*dij[1])
G_filtered = uniform_filter(G, size=roi_size)
points1d_0 = np.array(np.unravel_index(np.argsort(G_filtered.flatten())[-N_tracking_points:], G_filtered.shape)).T

fig, ax = plt.subplots()
ax.imshow(G_filtered, cmap='gray')
plt.show()

video.set_points(points1d_0)
video.show_points()
displacements_1d0 = video.get_displacements(processes=1, autosave=False)

We can also determine the motion perpendicular to $d$

In [ ]:
dij_T = (-dij[1], dij[0]) # horizontal direction
roi_size = (7, 3)
video.method.configure(reference_image = (0, 100), dij=dij_T, resume_analysis = False, tol=tol,roi_size=roi_size)

G = np.abs(Gi*dij_T[0] + Gj*dij_T[1])
G_filtered = uniform_filter(G, size=roi_size)
points1d_1 = np.array(np.unravel_index(np.argsort(G_filtered.flatten())[-N_tracking_points:], G_filtered.shape)).T

fig, ax = plt.subplots()
ax.imshow(G_filtered, cmap='gray')
plt.show()

video.set_points(points1d_1)
video.show_points()
displacements_1d1 = video.get_displacements(processes=1, autosave=False)

Let's compare results. In this synthethic case the motion is equal everywhere in the video, so the two orthogonal 1D motions can be added. 

In [ ]:
%matplotlib inline
displacements_1d = displacements_1d0 + displacements_1d1

t_vec = np.arange(0, len(displacements_1d[0])) * video.info['Record Rate(fps)']

fig, ax = plt.subplots(3, 1)
ax[2].set_xlabel('Time (s)')
ax[0].set_ylabel('x (pixels) - x')
ax[1].set_ylabel('y (pixels) -y')
ax[2].set_ylabel('Absolute error (pixels)')
ax[0].plot(t_vec, displacements_1d[0, :,0], 'b--', label = '1D - x')
ax[1].plot(t_vec, displacements_1d[0, :,1], 'b--', label = '1D - y')
ax[0].plot(t_vec, displacements_2d[0, :,0], 'r-', label = '2D - x')
ax[1].plot(t_vec, displacements_2d[0, :,1], 'r-', label = '2D - y')
ax[2].plot(t_vec, np.linalg.norm(displacements_2d[0]-displacements_1d[0], axis=1), 'k', label = 'absolute error')
ax[2].set_ylim(0, 0.02)
ax[0].legend()
ax[1].legend()
plt.show()

Watch the 1D motions

In [28]:
%matplotlib qt
td1d_0      = displacements_1d0 +  points1d_0.reshape(len(points1d_0),1,2)
td1d_1      = displacements_1d1 +  points1d_1.reshape(len(points1d_1),1,2)
combined_td = np.concatenate((td1d_0, td1d_1), axis=0)
ani         = play_video(range(1,video.reader.N), points=combined_td)

and compare the 2D motion extracted with conventional and Directional DIC

In [29]:
%matplotlib qt
td2d        = displacements_2d +  points2d.reshape(len(points2d),1,2)
td1d        = displacements_1d +  points2d.reshape(len(points2d),1,2)
combined_td = np.concatenate((td2d, td1d), axis=0)
ani         = play_video(range(1,video.reader.N), points=combined_td)

Directional DIC should be used with care. When the pixel intensity gradient is not aligned with $d$, accuracy is reduced

In [ ]:
%matplotlib inline
dij = (1, 0.1) # Close to vertical direction
roi_size = (3, 7) 
video.method.configure(reference_image = (0, 100), dij=dij, resume_analysis = False, tol=tol,roi_size=roi_size)
video.set_points(points1d_0)
displacements_1d2 = video.get_displacements(processes=1, autosave=False)

dij_T = (-dij[1], dij[0]) # Close to horizontal direction
roi_size = (7, 3)
video.method.configure(reference_image = (0, 100), dij=dij_T, resume_analysis = False, tol=tol,roi_size=roi_size)
video.set_points(points1d_1)
displacements_1d3 = video.get_displacements(processes=1, autosave=False)
displacements_1d2 = displacements_1d2 + displacements_1d3


fig, ax = plt.subplots(3, 1)
ax[2].set_xlabel('Time (s)')
ax[0].set_ylabel('x (pixels) - x')
ax[1].set_ylabel('y (pixels) -y')
ax[2].set_ylabel('Absolute error (pixels)')
ax[0].plot(t_vec, displacements_1d2[0, :,0], 'b--', label = '1D - x')
ax[1].plot(t_vec, displacements_1d2[0, :,1], 'b--', label = '1D - y')
ax[0].plot(t_vec, displacements_2d[0, :,0], 'r-', label = '2D - x')
ax[1].plot(t_vec, displacements_2d[0, :,1], 'r-', label = '2D - y')
ax[2].plot(t_vec, np.linalg.norm(displacements_2d[0]-displacements_1d2[0], axis=1), 'k', label = 'absolute error')
ax[2].set_ylim(0, 0.02)
ax[0].legend()
ax[1].legend()
plt.show()